In [ ]:
!pip install kagglehub
!pip install transformers
!pip install datasets
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("joykimaiyo18/linkedin-data-jobs-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/linkedin-data-jobs-dataset


In [ ]:
import pandas as pd
data = pd.read_csv(path + '/clean_jobs.csv')
data = data[["title","company","location","description"]]
data

,title,company,location,description
0,Data Analyst,Meta,"New York, NY",The Social Measurement team is a growing team ...
1,Data Analyst,Meta,"San Francisco, CA",The Social Measurement team is a growing team ...
2,Data Analyst,Meta,"Los Angeles, CA",The Social Measurement team is a growing team ...
3,Data Analyst,Meta,"Washington, DC",The Social Measurement team is a growing team ...
4,Data Analyst II,Pinterest,"Chicago, IL",About Pinterest\n\nMillions of people around t...
...,...,...,...,...
322,Data Engineer- Python Pyspark,Virtusa,"Chennai, Tamil Nadu, India",Senior Data Engineer\n\nPosition Summary\n\nTh...
323,Data Engineer with Pyspark,Cognizant,"Bangalore Urban, Karnataka, India",Job Title:- Data Engineer with Pyspark\n\nLoca...
324,Data Engineer,Mercedes-Benz Malaysia,"Puchong, Selangor, Malaysia","About Us\n\n\n\n\nAt Mercedes-Benz, we don’t j..."
325,Data Engineer I,IntePros,"Seattle, WA",Data Engineer I – Infrastructure & Automation ...


In [ ]:
# prompt: I would like to split the "data" dataframe in three hugging face dataset (train 80% of the data, validation 10% and test 10% stratified by title and previously randomly shuffled), how do I od that?

from sklearn.model_selection import train_test_split
from datasets import Dataset

# Assuming 'data' DataFrame is already defined and loaded as in the previous code

# Shuffle the data randomly
data_shuffled = data.sample(frac=1, random_state=42).reset_index(drop=True)
company_counts = data_shuffled['title'].value_counts()
less_frequent_companies = company_counts[company_counts < 5].index
data_shuffled.loc[data_shuffled['title'].isin(less_frequent_companies), 'title'] = 'Other'

# Split the data into train, validation, and test sets stratified by 'company'
train_data, temp_data = train_test_split(
    data_shuffled, test_size=0.2, random_state=42,stratify=data_shuffled['title']
)
val_data, test_data = train_test_split(
    temp_data, test_size=0.5, random_state=42,stratify=temp_data['title']
)
del temp_data

# Convert pandas DataFrames to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)
test_dataset = Dataset.from_pandas(test_data)

# Print the sizes of the datasets
print("Train dataset size:", len(train_dataset))
print("Validation dataset size:", len(val_dataset))
print("Test dataset size:", len(test_dataset))


Train dataset size: 261
Validation dataset size: 33
Test dataset size: 33


In [ ]:
pd.Series(train_dataset["title"]).value_counts()

,count
Other,134
Data Analyst,42
Data Engineer,23
"Data Scientist, Product Analytics",21
Data Scientist,19
"Data Engineer, Product Analytics",15
Machine Learning Engineer,7


In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification
unique_titles = pd.Series(pd.concat([pd.Series(train_dataset["title"]), pd.Series(val_dataset["title"])])).unique()
label2id = {title: id for id, title in enumerate(unique_titles)}
id2label = {id: title for title, id in label2id.items()}
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("distilbert/distilbert-base-uncased", num_labels=len(unique_titles), id2label=id2label,label2id=label2id)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# prompt: can you fine tune this model on the train dataset on 10 epochs

import numpy as np
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score
from torch.optim import AdamW
# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["description"], padding="max_length", truncation=True)

# Tokenize the datasets
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_train_dataset = tokenized_train_dataset.rename_column("title", "labels")
tokenized_train_dataset = tokenized_train_dataset.rename_column("description", "text")
# Convert labels to numerical IDs using label2id
tokenized_train_dataset = tokenized_train_dataset.map(lambda examples: {'labels': [label2id[label] for label in examples['labels']]}, batched=True) # This line was added to convert the labels to numerical IDs.
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = tokenized_val_dataset.rename_column("title", "labels")
tokenized_val_dataset = tokenized_val_dataset.rename_column("description", "text")
# Convert labels to numerical IDs using label2id
tokenized_val_dataset = tokenized_val_dataset.map(lambda examples: {'labels': [label2id[label] for label in examples['labels']]}, batched=True) # This line was added to convert the labels to numerical IDs.

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    load_best_model_at_end=True,
    push_to_hub=False,
    metric_for_best_model="accuracy",
    report_to="none"
)
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc}
# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    compute_metrics=compute_metrics
)

# Fine-tune the model
trainer.train()


Map:   0%|          | 0/261 [00:00<?, ? examples/s]

Map:   0%|          | 0/261 [00:00<?, ? examples/s]

Map:   0%|          | 0/33 [00:00<?, ? examples/s]

Map:   0%|          | 0/33 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.364892,0.606061
2,No log,1.098364,0.606061
3,No log,0.916457,0.727273
4,No log,0.832884,0.727273
5,No log,0.837830,0.757576
6,No log,0.651531,0.818182
7,No log,0.510821,0.787879
8,No log,0.462068,0.909091
9,No log,0.427964,0.939394
10,No log,0.433085,0.939394


TrainOutput(global_step=170, training_loss=0.6701471665326286, metrics={'train_runtime': 194.14, 'train_samples_per_second': 13.444, 'train_steps_per_second': 0.876, 'total_flos': 345770739394560.0, 'train_loss': 0.6701471665326286, 'epoch': 10.0})

In [ ]:
M=2000/8
lambd = 5e-3
eta = 5e-3
opt = 1/(M*lambd*eta)
print(opt)
newM=30000/8

newwd = 1/(newM*opt*eta)
print(eta,newwd)

160.0
0.005 0.0003333333333333333


In [ ]:
import itertools
dico={}
for lr,wd in itertools.product([5e-3,1e-3,5e-4],[0.0003333333333333,5e-3,1e-3]):
  # prompt: now reinitialized the same base model and add lora to fine tune more efficiently

  import torch.nn as nn
  import peft
  from peft import AdaLoraConfig, get_peft_model, TaskType, PeftType
  model = AutoModelForSequenceClassification.from_pretrained("distilbert/distilbert-base-uncased", num_labels=len(unique_titles), id2label=id2label,label2id=label2id)
  supported_modules = (nn.Linear, nn.Conv2d, nn.Conv3d)
  target_modules = [
      name for name, module in model.named_modules()
      if isinstance(module, supported_modules) and not "classifier" in name and name != "" and not "pre_classifier" in name
  ]
  exclude_modules=[
      name for name, module in model.named_modules()
      if isinstance(module, supported_modules) and ("classifier" in name or "pre_classifier" in name) and name != ""
  ]
  print(target_modules)
  peft_config = AdaLoraConfig(task_type=TaskType.SEQ_CLS,
                              peft_type=PeftType.ADALORA,
                              inference_mode=False,
                              total_step=2,
                              use_rslora=True,
                              modules_to_save=[],
                              target_r=4,
                              init_r=6,
                              lora_dropout=0.1,
                              target_modules=target_modules,
                              exclude_modules=exclude_modules)
  model = get_peft_model(model, peft_config)

  model.print_trainable_parameters()
  model


  import numpy as np
  from transformers import Trainer, TrainingArguments
  from sklearn.metrics import accuracy_score
  # Tokenization function
  def tokenize_function(examples):
      return tokenizer(examples["description"], padding="max_length", truncation=True)

  # Tokenize the datasets
  tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
  tokenized_train_dataset = tokenized_train_dataset.rename_column("title", "labels")
  tokenized_train_dataset = tokenized_train_dataset.rename_column("description", "text")
  # Convert labels to numerical IDs using label2id
  tokenized_train_dataset = tokenized_train_dataset.map(lambda examples: {'labels': [label2id[label] for label in examples['labels']]}, batched=True) # This line was added to convert the labels to numerical IDs.
  tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)
  tokenized_val_dataset = tokenized_val_dataset.rename_column("title", "labels")
  tokenized_val_dataset = tokenized_val_dataset.rename_column("description", "text")
  # Convert labels to numerical IDs using label2id
  tokenized_val_dataset = tokenized_val_dataset.map(lambda examples: {'labels': [label2id[label] for label in examples['labels']]}, batched=True) # This line was added to convert the labels to numerical IDs.
  def compute_metrics(pred):
      labels = pred.label_ids
      preds = pred.predictions.argmax(-1)
      acc = accuracy_score(labels, preds)
      return {"accuracy": acc}
  # Redefine the Trainer with the PEFT model
  training_args = TrainingArguments(
      output_dir="./results",
      eval_strategy="epoch",
      save_strategy="epoch",
      learning_rate=lr,
      per_device_train_batch_size=8,
      per_device_eval_batch_size=8,
      num_train_epochs=2,
      weight_decay=wd,
      load_best_model_at_end=True,
      push_to_hub=False,
      metric_for_best_model="accuracy",
      report_to="none"
  )

  trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=tokenized_train_dataset,
      eval_dataset=tokenized_val_dataset,
      compute_metrics=compute_metrics
  )

  # Fine-tune the model
  trainer.train()

  # prompt: how to test the performance of the fine tuned model on the test dataset?

  # Tokenize the test dataset
  tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)
  tokenized_test_dataset = tokenized_test_dataset.rename_column("title", "labels")
  tokenized_test_dataset = tokenized_test_dataset.rename_column("description", "text")
  # Convert labels to numerical IDs using label2id
  tokenized_test_dataset = tokenized_test_dataset.map(lambda examples: {'labels': [label2id[label] for label in examples['labels']]}, batched=True)

  # Evaluate the model on the test dataset
  results = trainer.predict(tokenized_test_dataset)

  # Get predictions and true labels
  predicted_labels = np.argmax(results.predictions, axis=1)
  true_labels = results.label_ids

  # Calculate accuracy on the test set
  test_accuracy = accuracy_score(true_labels, predicted_labels)
  print(f"Test Accuracy: {test_accuracy}")
  dico[f"lr={lr},wd={wd}"]=test_accuracy
  # You can also calculate other metrics as needed, e.g., precision, recall, F1-score
  # using sklearn.metrics functions
dico

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


['distilbert.transformer.layer.0.attention.q_lin', 'distilbert.transformer.layer.0.attention.k_lin', 'distilbert.transformer.layer.0.attention.v_lin', 'distilbert.transformer.layer.0.attention.out_lin', 'distilbert.transformer.layer.0.ffn.lin1', 'distilbert.transformer.layer.0.ffn.lin2', 'distilbert.transformer.layer.1.attention.q_lin', 'distilbert.transformer.layer.1.attention.k_lin', 'distilbert.transformer.layer.1.attention.v_lin', 'distilbert.transformer.layer.1.attention.out_lin', 'distilbert.transformer.layer.1.ffn.lin1', 'distilbert.transformer.layer.1.ffn.lin2', 'distilbert.transformer.layer.2.attention.q_lin', 'distilbert.transformer.layer.2.attention.k_lin', 'distilbert.transformer.layer.2.attention.v_lin', 'distilbert.transformer.layer.2.attention.out_lin', 'distilbert.transformer.layer.2.ffn.lin1', 'distilbert.transformer.layer.2.ffn.lin2', 'distilbert.transformer.layer.3.attention.q_lin', 'distilbert.transformer.layer.3.attention.k_lin', 'distilbert.transformer.layer.3.att

Map:   0%|          | 0/261 [00:00<?, ? examples/s]

Map:   0%|          | 0/261 [00:00<?, ? examples/s]

Map:   0%|          | 0/33 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/261 [00:00<?, ? examples/s]

Map:   0%|          | 0/33 [00:00<?, ? examples/s]

Map:   0%|          | 0/33 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.447376,0.515152


In [ ]:
import torch.nn as nn
import peft
from peft import AdaLoraConfig, get_peft_model, TaskType, PeftType
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
from sklearn.metrics import balanced_accuracy_score, f1_score
from torch.optim import AdamW
model = AutoModelForSequenceClassification.from_pretrained("distilbert/distilbert-base-uncased", num_labels=len(unique_titles), id2label=id2label,label2id=label2id)
supported_modules = (nn.Linear, nn.Conv2d, nn.Conv3d)
target_modules = [
    name for name, module in model.named_modules()
    if isinstance(module, supported_modules) and not "classifier" in name and name != "" and not "pre_classifier" in name
]
exclude_modules=[
    name for name, module in model.named_modules()
    if isinstance(module, supported_modules) and ("classifier" in name or "pre_classifier" in name) and name != ""
]
print(target_modules)
peft_config = AdaLoraConfig(task_type=TaskType.SEQ_CLS,
                            peft_type=PeftType.ADALORA,
                            inference_mode=False,
                            total_step=2,
                            use_rslora=True,
                            modules_to_save=[],
                            target_r=4,
                            init_r=6,
                            lora_dropout=0.1,
                            target_modules=target_modules,
                            exclude_modules=exclude_modules)
model = get_peft_model(model, peft_config)

model.print_trainable_parameters()
model


import numpy as np
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score
# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["description"], padding="max_length", truncation=True)

# Tokenize the datasets
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_train_dataset = tokenized_train_dataset.rename_column("title", "labels")
tokenized_train_dataset = tokenized_train_dataset.rename_column("description", "text")
#tokenized_train_dataset = tokenized_train_dataset.rename_column("label", "labels")
# Convert labels to numerical IDs using label2id
tokenized_train_dataset = tokenized_train_dataset.map(lambda examples: {'labels': [label2id[label] for label in examples['labels']]}, batched=True) # This line was added to convert the labels to numerical IDs.
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = tokenized_val_dataset.rename_column("title", "labels")
tokenized_val_dataset = tokenized_val_dataset.rename_column("description", "text")
#tokenized_val_dataset = tokenized_val_dataset.rename_column("label", "labels")
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc}
optimizer = AdamW(
filter(lambda p: p.requires_grad, model.parameters()),
lr=5e-3,
weight_decay=0.001
)
train_dataloader = DataLoader(
tokenized_train_dataset,
batch_size=8,
shuffle=True
)
eval_dataloader = DataLoader(
tokenized_val_dataset,
batch_size=8,
shuffle=True
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
nb_batch = len(train_dataloader)
bacallback=0.0
callback=0
patience=0
epochs=10
for epoch in range(epochs):  # Number of epochs
  callback+=1
  patience+=1
  model.train()
  total_loss = 0
  labeltrain=torch.tensor([]).to(device)
  predtrain=torch.tensor([]).to(device)

  for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{epochs}"):
      # Move batch to device
      # Since the DataLoader returns a dictionary of lists of tensors,
      # we need to process each item in the dictionary.
      # Labels should not be squeezed.
      processed_batch = {}
      for k, v in batch.items():
          if k == 'labels':
              processed_batch[k] = v.to(device)
          else:
              # Assuming other items are sequences of tensors
              processed_batch[k] = v.to(device).squeeze()

      labeltrain=torch.cat((labeltrain,processed_batch["labels"].cpu()))
      # Forward pass
      outputs = model(**processed_batch)
      loss = outputs.loss
      predtrain=torch.cat((predtrain,torch.argmax(outputs.logits,dim=1).cpu()))
      # Backward pass
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()

      total_loss += loss.item()

      batrain = balanced_accuracy_score(labeltrain.numpy(),predtrain.numpy())
      model.eval()
      labelval = torch.tensor([]).to(device)
      predval = torch.tensor([]).to(device)
      with torch.no_grad():
          for batch in eval_dataloader:
              # Process the batch for evaluation as well
              processed_batch_eval = {}
              for k, v in batch.items():
                  if k == 'labels':
                      processed_batch_eval[k] = v.to(device)
                  else:
                      processed_batch_eval[k] = v.to(device).squeeze()

              outputs = model(**processed_batch_eval)

              labelval = torch.cat((labelval, processed_batch_eval["labels"].cpu()))
              predval = torch.cat((predval, torch.argmax(outputs.logits, dim=1).cpu()))
      baval = f1_score(labelval.numpy(), predval.numpy(),average="weighted")
      if patience<=2:
          callback=0
      if baval > bacallback:
          bacallback=baval
          bestmodel = model.state_dict()
          callback=0
      print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_dataloader)}, Train BA: {batrain}, Val BA: {baval}")
      if callback >= 3:
          break

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


['distilbert.transformer.layer.0.attention.q_lin', 'distilbert.transformer.layer.0.attention.k_lin', 'distilbert.transformer.layer.0.attention.v_lin', 'distilbert.transformer.layer.0.attention.out_lin', 'distilbert.transformer.layer.0.ffn.lin1', 'distilbert.transformer.layer.0.ffn.lin2', 'distilbert.transformer.layer.1.attention.q_lin', 'distilbert.transformer.layer.1.attention.k_lin', 'distilbert.transformer.layer.1.attention.v_lin', 'distilbert.transformer.layer.1.attention.out_lin', 'distilbert.transformer.layer.1.ffn.lin1', 'distilbert.transformer.layer.1.ffn.lin2', 'distilbert.transformer.layer.2.attention.q_lin', 'distilbert.transformer.layer.2.attention.k_lin', 'distilbert.transformer.layer.2.attention.v_lin', 'distilbert.transformer.layer.2.attention.out_lin', 'distilbert.transformer.layer.2.ffn.lin1', 'distilbert.transformer.layer.2.ffn.lin2', 'distilbert.transformer.layer.3.attention.q_lin', 'distilbert.transformer.layer.3.attention.k_lin', 'distilbert.transformer.layer.3.att

Map:   0%|          | 0/261 [00:00<?, ? examples/s]

Map:   0%|          | 0/261 [00:00<?, ? examples/s]

Map:   0%|          | 0/33 [00:00<?, ? examples/s]

Epoch 1/10:   0%|          | 0/33 [00:00<?, ?it/s]


AttributeError: 'list' object has no attribute 'to'

In [ ]:
import torch.nn as nn
import peft
from peft import AdaLoraConfig, get_peft_model, TaskType, PeftType
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
from sklearn.metrics import balanced_accuracy_score, f1_score
from torch.optim import AdamW
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import numpy as np # Make sure numpy is imported

# Ensure tokenizer is defined if not already
# from transformers import AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

model = AutoModelForSequenceClassification.from_pretrained("distilbert/distilbert-base-uncased", num_labels=len(unique_titles), id2label=id2label,label2id=label2id)
supported_modules = (nn.Linear, nn.Conv2d, nn.Conv3d)
target_modules = [
    name for name, module in model.named_modules()
    if isinstance(module, supported_modules) and not "classifier" in name and name != "" and not "pre_classifier" in name
]
exclude_modules=[
    name for name, module in model.named_modules()
    if isinstance(module, supported_modules) and ("classifier" in name or "pre_classifier" in name) and name != ""
]
print(target_modules)
peft_config = AdaLoraConfig(task_type=TaskType.SEQ_CLS,
                            peft_type=PeftType.ADALORA,
                            inference_mode=False,
                            total_step=2,
                            use_rslora=True,
                            modules_to_save=[],
                            target_r=4,
                            init_r=6,
                            lora_dropout=0.1,
                            target_modules=target_modules,
                            exclude_modules=exclude_modules)
model = get_peft_model(model, peft_config)

model.print_trainable_parameters()
model

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["description"], padding="max_length", truncation=True)

# Tokenize the datasets
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_train_dataset = tokenized_train_dataset.rename_column("title", "labels")
tokenized_train_dataset = tokenized_train_dataset.rename_column("description", "text")
#tokenized_train_dataset = tokenized_train_dataset.rename_column("label", "labels")
# Convert labels to numerical IDs using label2id
tokenized_train_dataset = tokenized_train_dataset.map(lambda examples: {'labels': [label2id[label] for label in examples['labels']]}, batched=True) # This line was added to convert the labels to numerical IDs.
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = tokenized_val_dataset.rename_column("title", "labels")
tokenized_val_dataset = tokenized_val_dataset.rename_column("description", "text")
#tokenized_val_dataset = tokenized_val_dataset.rename_column("label", "labels")

# Remove columns that are not needed by the model forward pass to avoid issues with DataLoader
columns_to_return = ['input_ids', 'attention_mask', 'labels']
tokenized_train_dataset.set_format(type='torch', columns=columns_to_return)
tokenized_val_dataset.set_format(type='torch', columns=columns_to_return)


def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc}

optimizer = AdamW(
filter(lambda p: p.requires_grad, model.parameters()),
lr=5e-3,
weight_decay=0.001
)

train_dataloader = DataLoader(
tokenized_train_dataset,
batch_size=8,
shuffle=True
)
eval_dataloader = DataLoader(
tokenized_val_dataset,
batch_size=8,
shuffle=False # Shuffle should be False for evaluation to ensure order
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

nb_batch = len(train_dataloader)
bacallback=0.0
callback=0
patience=0
epochs=10

for epoch in range(epochs):  # Number of epochs
  callback+=1
  patience+=1
  model.train()
  total_loss = 0
  labeltrain_list = [] # Use lists to accumulate CPU tensors
  predtrain_list = []

  for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{epochs}"):
      # Move batch to device
      batch = {k: v.to(device) for k, v in batch.items()}

      # Forward pass
      outputs = model(**batch)
      loss = outputs.loss

      # Accumulate labels and predictions on CPU
      labeltrain_list.append(batch["labels"].cpu())
      predtrain_list.append(torch.argmax(outputs.logits, dim=1).cpu())

      # Backward pass
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()

      total_loss += loss.item()

  # Compute training metrics for the epoch after processing all batches
  labeltrain_epoch = torch.cat(labeltrain_list).numpy()
  predtrain_epoch = torch.cat(predtrain_list).numpy()
  batrain_epoch = balanced_accuracy_score(labeltrain_epoch, predtrain_epoch)

  # Calculate average training loss per batch for the epoch
  avg_train_loss = total_loss / len(train_dataloader)


  model.eval()
  labelval_list = [] # Use lists to accumulate CPU tensors
  predval_list = []
  with torch.no_grad():
      for batch in eval_dataloader:
          # Move batch to device
          batch = {k: v.to(device) for k, v in batch.items()}

          # Forward pass
          outputs = model(**batch)

          # Accumulate labels and predictions on CPU
          labelval_list.append(batch["labels"].cpu())
          predval_list.append(torch.argmax(outputs.logits, dim=1).cpu())

  # Compute validation metrics for the epoch after processing all batches
  labelval_epoch = torch.cat(labelval_list).numpy()
  predval_epoch = torch.cat(predval_list).numpy()
  baval_epoch = f1_score(labelval_epoch, predval_epoch, average="weighted")

  if patience <= 2:
      callback = 0
  if baval_epoch > bacallback:
      bacallback = baval_epoch
      bestmodel = model.state_dict()
      callback = 0

  print(f"Epoch {epoch+1}, Loss: {avg_train_loss}, Train BA: {batrain_epoch}, Val BA: {baval_epoch}")

  if callback >= 3:
      print("Early stopping triggered.")
      break

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


['distilbert.transformer.layer.0.attention.q_lin', 'distilbert.transformer.layer.0.attention.k_lin', 'distilbert.transformer.layer.0.attention.v_lin', 'distilbert.transformer.layer.0.attention.out_lin', 'distilbert.transformer.layer.0.ffn.lin1', 'distilbert.transformer.layer.0.ffn.lin2', 'distilbert.transformer.layer.1.attention.q_lin', 'distilbert.transformer.layer.1.attention.k_lin', 'distilbert.transformer.layer.1.attention.v_lin', 'distilbert.transformer.layer.1.attention.out_lin', 'distilbert.transformer.layer.1.ffn.lin1', 'distilbert.transformer.layer.1.ffn.lin2', 'distilbert.transformer.layer.2.attention.q_lin', 'distilbert.transformer.layer.2.attention.k_lin', 'distilbert.transformer.layer.2.attention.v_lin', 'distilbert.transformer.layer.2.attention.out_lin', 'distilbert.transformer.layer.2.ffn.lin1', 'distilbert.transformer.layer.2.ffn.lin2', 'distilbert.transformer.layer.3.attention.q_lin', 'distilbert.transformer.layer.3.attention.k_lin', 'distilbert.transformer.layer.3.att

Map:   0%|          | 0/261 [00:00<?, ? examples/s]

Map:   0%|          | 0/261 [00:00<?, ? examples/s]

Map:   0%|          | 0/33 [00:00<?, ? examples/s]

Epoch 1/10:   0%|          | 0/33 [00:00<?, ?it/s]


ValueError: Unable to avoid copy while creating an array as requested.
If using `np.array(obj, copy=False)` replace it with `np.asarray(obj)` to allow a copy when needed (no behavior change in NumPy 1.x).
For more details, see https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword.